<a href="https://colab.research.google.com/github/Prajna1999/eng-or-mt/blob/master/tokenizer_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers sentencepiece

In [4]:
!pip install --upgrade huggingface_hub


In [8]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=False)
print(tokenizer.bos_token)

<s>


In [25]:
import unicodedata
class TokenizerWrapper:
    def __init__(self, base_tokenizer):
      self.tokenizer=base_tokenizer

    def normalize_unicode(self, text):
      normalized_text=unicodedata.normalize("NFKC", text)
      return normalized_text

    def encode(self, text, **kwargs):
      normalized=self.normalize_unicode(text)
      # call the encoded text
      encoded=self.tokenizer.encode(normalized, **kwargs)
      return encoded

In [26]:
wrapper=TokenizerWrapper(tokenizer)

In [27]:
def log_tokenization_diff(wrapper, raw_text):
    from pprint import pprint

    print("📝 Original Text:", raw_text)

    # Tokenize raw (non-normalized) text
    raw_tokens = tokenizer.tokenize(raw_text)
    raw_ids = tokenizer.encode(raw_text, add_special_tokens=False)

    print("\n🔍 Tokenization WITHOUT normalization:")
    pprint(raw_tokens)
    print("Token IDs:", raw_ids)

    # Normalize + tokenize
    normalized_text = wrapper.normalize_unicode(raw_text)
    norm_tokens = tokenizer.tokenize(normalized_text)
    norm_ids = tokenizer.encode(normalized_text, add_special_tokens=False)

    print("\n✨ Tokenization WITH NFC normalization:")
    pprint(norm_tokens)
    print("Token IDs:", norm_ids)

    # Bonus: show graphemes if useful
    import regex
    graphemes = regex.findall(r"\X", normalized_text)
    print("\n🧩 Grapheme Clusters after normalization:")
    pprint(graphemes)


In [30]:
text =  "ଜ୍ଞାନ"
log_tokenization_diff(wrapper, text)


📝 Original Text: ଜ୍ଞାନ

🔍 Tokenization WITHOUT normalization:
['▁',
 '<0xE0>',
 '<0xAC>',
 '<0x9C>',
 '<0xE0>',
 '<0xAD>',
 '<0x8D>',
 '<0xE0>',
 '<0xAC>',
 '<0x9E>',
 '<0xE0>',
 '<0xAC>',
 '<0xBE>',
 '<0xE0>',
 '<0xAC>',
 '<0xA8>']
Token IDs: [29473, 995, 943, 927, 995, 944, 912, 995, 943, 929, 995, 943, 961, 995, 943, 939]

✨ Tokenization WITH NFC normalization:
['▁',
 '<0xE0>',
 '<0xAC>',
 '<0x9C>',
 '<0xE0>',
 '<0xAD>',
 '<0x8D>',
 '<0xE0>',
 '<0xAC>',
 '<0x9E>',
 '<0xE0>',
 '<0xAC>',
 '<0xBE>',
 '<0xE0>',
 '<0xAC>',
 '<0xA8>']
Token IDs: [29473, 995, 943, 927, 995, 944, 912, 995, 943, 929, 995, 943, 961, 995, 943, 939]

🧩 Grapheme Clusters after normalization:
['ଜ୍ଞା', 'ନ']


In [21]:
# without pre-processing
encoded=tokenizer.encode(text)
print(f"output is {encoded} and total tokens are: {len(encoded)}")

output is [1, 29473, 995, 943, 920, 995, 944, 912, 995, 943, 954, 995, 943, 945, 995, 943, 961, 29473, 995, 943, 920, 995, 943, 947, 995, 943, 939, 995, 944, 912, 995, 943, 935, 995, 944, 900] and total tokens are: 36
